In [1]:
import pandas as pd 
import numpy as np
from scipy.optimize import linprog 

In [2]:
data = pd.read_csv('data.csv', sep=';')

In [3]:
data.head(5)

,Product Name,Production Unit Name,Currency,Production Plan,Planned Sale price,Price - Model Curr(USD),Material Cost,Marginal profit per unit,Total marginal profit,Min production lot restrictions,Contract minimum,Market maximum
0,"Candies ""Hazelnuts"". 0.25 kg",Candies,TRY,8200.0,31.406,2.293,1.335,0.958,7856.980,100,100,15000
1,"Candies ""Hazelnuts"". 0.25 kg",Candies,EUR,100.0,2.021,2.284,1.335,0.950,94.959,100,0,10000
2,"Candies ""Hazelnuts"". 0.25 kg",Candies,USD,100.0,2.283,2.283,1.335,0.948,94.834,100,100,5000
3,"Candies ""Cashews"". 0.25 kg",Candies,TRY,1500.0,45.777,3.342,1.925,1.417,2125.330,100,150,1500
4,"Candies ""Cashews"". 0.25 kg",Candies,EUR,800.0,2.946,3.329,1.925,1.404,1123.500,100,0,800


In [4]:
sum_total_marginal_profit = sum(data['Total marginal profit'])
print(sum_total_marginal_profit)

38158.94800000001


In [5]:
data.dtypes

Product Name                        object
Production Unit Name                object
Currency                            object
Production Plan                    float64
Planned Sale price                 float64
Price -  Model Curr(USD)           float64
Material Cost                      float64
Marginal profit per unit           float64
Total marginal profit              float64
Min production lot restrictions      int64
Contract minimum                     int64
Market maximum                       int64
dtype: object

In [6]:
sum_candies = sum(data[data['Production Unit Name'] == 'Candies']['Production Plan'])
sum_lolipops = sum(data[data['Production Unit Name'] == 'Lollipops']['Production Plan'])
sum_dragee = sum(data[data['Production Unit Name'] == 'Dragee']['Production Plan'])

In [7]:
sum_candies, sum_lolipops, sum_dragee

(12000.0, 5000.0, 9999.996000000001)

In [8]:
data_1 = data[data['Production Unit Name'] == 'Candies']
MarginPPU = data_1['Marginal profit per unit']

In [9]:
data_1

,Product Name,Production Unit Name,Currency,Production Plan,Planned Sale price,Price - Model Curr(USD),Material Cost,Marginal profit per unit,Total marginal profit,Min production lot restrictions,Contract minimum,Market maximum
0,"Candies ""Hazelnuts"". 0.25 kg",Candies,TRY,8200.0,31.406,2.293,1.335,0.958,7856.980,100,100,15000
1,"Candies ""Hazelnuts"". 0.25 kg",Candies,EUR,100.0,2.021,2.284,1.335,0.950,94.959,100,0,10000
2,"Candies ""Hazelnuts"". 0.25 kg",Candies,USD,100.0,2.283,2.283,1.335,0.948,94.834,100,100,5000
3,"Candies ""Cashews"". 0.25 kg",Candies,TRY,1500.0,45.777,3.342,1.925,1.417,2125.330,100,150,1500
4,"Candies ""Cashews"". 0.25 kg",Candies,EUR,800.0,2.946,3.329,1.925,1.404,1123.500,100,0,800
5,"Candies ""Cashews"". 0.25 kg",Candies,USD,1000.0,3.327,3.327,1.925,1.403,1402.560,100,200,1000
6,"Candies ""Pistachios"". 0.25 kg",Candies,TRY,100.0,48.719,3.557,3.307,0.249,24.944,100,50,1500
7,"Candies ""Pistachios"". 0.25 kg",Candies,EUR,100.0,3.136,3.543,3.307,0.236,23.613,100,0,900
8,"Candies ""Pistachios"". 0.25 kg",Candies,USD,100.0,3.541,3.541,3.307,0.234,23.419,100,50,2000


In [29]:
c = -data_1['Production Plan']
c = pd.to_numeric(c)

In [11]:
B = []
for b in range(len(data_1)):
    B.append(max(data_1['Min production lot restrictions'][b], data_1['Contract minimum'][b]))
B = pd.Series(B)

bounds = [(b, None) for b in B]

In [12]:
-MarginPPU

0   -0.958
1   -0.950
2   -0.948
3   -1.417
4   -1.404
5   -1.403
6   -0.249
7   -0.236
8   -0.234
Name: Marginal profit per unit, dtype: float64

In [13]:
A_ub = np.diag(MarginPPU)

In [20]:
A_ub

array([[0.958, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.95 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.948, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 1.417, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 1.404, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 1.403, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.249, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.236, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.234]])

In [14]:
b_ub = np.array(data_1['Market maximum'])

In [21]:
b_ub

array([15000, 10000,  5000,  1500,   800,  1000,  1500,   900,  2000],
      dtype=int64)

In [15]:
A_eq = np.array(MarginPPU).reshape(1, -1)

In [22]:
A_eq

array([[0.958, 0.95 , 0.948, 1.417, 1.404, 1.403, 0.249, 0.236, 0.234]])

In [16]:
b_eq = 12000

In [24]:
bounds

[(100, None),
 (100, None),
 (100, None),
 (150, None),
 (100, None),
 (200, None),
 (100, None),
 (100, None),
 (100, None)]

In [30]:
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

In [31]:
res

     con: array([2.71047793e-06])
     fun: -95606096.01017796
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([3895.25000271, 9905.        , 4905.2       , 1287.45      ,
        659.6       ,  719.4       , 1475.1       ,  876.4       ,
       1976.6       ])
  status: 0
 success: True
       x: array([11591.59707441,   100.        ,   100.        ,   150.        ,
         100.        ,   200.        ,   100.        ,   100.        ,
         100.        ])

In [19]:
sum([11591.59707724,   100.        ,   100.        ,   150.        ,
         100.        ,   200.        ,   100.        ,   100.        ,
         100.        ])

12541.59707724